In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("./")

### Q1. Install the package

In [3]:
!mlflow --version

mlflow, version 2.3.2


### Q2. Download and preprocess the data

In [55]:
!pwd

/Users/rguarise/Documents/mlops-zoomcamp/homeworks/02-experiment-tracking


In [58]:
!python ./utils/preprocess_data.py --raw_data_path $HOME/Documents/mlops-zoomcamp/data/ --dest_path ./output

In [59]:
!ls -l ./output

total 17320
-rw-r--r--  1 rguarise  1010544492   153660 Jun  1 21:41 dv.pkl
-rw-r--r--  1 rguarise  1010544492  2632817 Jun  1 21:41 test.pkl
-rw-r--r--  1 rguarise  1010544492  2146163 Jun  1 21:41 train.pkl
-rw-r--r--  1 rguarise  1010544492  2336393 Jun  1 21:41 val.pkl


### Q3. Train a model with autolog

In [5]:
import utils.train as train
train.run_train("./output/")

### Q4. Tune model hyperparameters

In [6]:
import utils.hpo as hpo
hpo.run_optimization("./output/",10)

2023/06/04 18:22:13 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
[I 2023-06-04 18:22:13,728] A new study created in memory with name: no-name-52f65d54-32bd-4e5c-a95d-2cae6c3941c2
[I 2023-06-04 18:22:15,403] Trial 0 finished with value: 2.451379690825458 and parameters: {'n_estimators': 25, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 0 with value: 2.451379690825458.
[I 2023-06-04 18:22:15,614] Trial 1 finished with value: 2.4667366020368333 and parameters: {'n_estimators': 16, 'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 0 with value: 2.451379690825458.
[I 2023-06-04 18:22:16,922] Trial 2 finished with value: 2.449827329704216 and parameters: {'n_estimators': 34, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 2 with value: 2.449827329704216.
[I 2023-06-04 18:22:17,613] Trial 3 finished with value: 2.460983516558473 a

### Q5. Promote the best model to the model registry

In [8]:
import utils.register_model as register_model
register_model.run_register_model("./output/",5)

Registered model 'best_model' already exists. Creating a new version of this model...
2023/06/04 18:25:45 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: best_model, version 2


test_rmse: 2.2913864757421787


Created version '2' of model 'best_model'.
